In [13]:
import pandas as pd
from sklearn.metrics import r2_score, accuracy_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from rustrees.rustrees import DecisionTree, RandomForest
import rustrees.tree as rt
import time
import numpy as np

In [14]:
datasets = {
    "reg": ["diabetes", "housing"],
    "clf": ["breast_cancer", "titanic"]
}

In [15]:
def evaluate_dataset(dataset, problem, model="dt", max_depth=5, n_estimators=10, n_repeats=100):
    df_train = pd.read_csv(f"../../datasets/{dataset}_train.csv")
    df_test = pd.read_csv(f"../../datasets/{dataset}_test.csv")
    
    df_train_rt = rt.from_pandas(df_train)
    df_test_rt = rt.from_pandas(df_test)

    if problem == "reg":
        metric_fn = r2_score
        metric = "r2"
        if model == "dt":
            model_sk = DecisionTreeRegressor(max_depth=max_depth)
        elif model == "rf":
            model_sk = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
    elif problem == "clf":
        metric_fn = accuracy_score
        metric = "acc"
        if model == "dt":
            model_sk = DecisionTreeClassifier(max_depth=max_depth)
        elif model == "rf":
            model_sk = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
    
    start_time = time.time()
    results_sk = []
    for _ in range(n_repeats):
        model_sk.fit(df_train.drop("target", axis=1), df_train.target)
        results_sk.append(metric_fn(df_test.target, model_sk.predict(df_test.drop("target", axis=1))))
    sk_time = (time.time() - start_time)/n_repeats
    sk_mean = np.mean(results_sk)
    sk_std = np.std(results_sk)
    
    start_time = time.time()
    results_rt = []
    for _ in range(n_repeats):
        if problem == "reg" and model == "dt":
            model_rt = DecisionTree.train_reg(df_train_rt, max_depth=max_depth)
        elif problem == "reg" and model == "rf":
            model_rt = RandomForest.train_reg(df_train_rt, n_estimators=n_estimators, max_depth=max_depth)
        elif problem == "clf" and model == "dt":
            model_rt = DecisionTree.train_clf(df_train_rt, max_depth=max_depth)   
        elif problem == "clf" and model == "rf":
            model_rt = RandomForest.train_clf(df_train_rt, n_estimators=n_estimators, max_depth=max_depth)

        if problem == "reg":
            pred_rt = model_rt.predict(df_test_rt)
            results_rt.append(metric_fn(df_test.target, pred_rt))
        elif problem == "clf":
            pred_rt = model_rt.predict(df_test_rt)
            results_rt.append(metric_fn(df_test.target, np.array(pred_rt) > 0.5))
    rt_time = (time.time() - start_time)/n_repeats
    rt_mean = np.mean(results_rt)
    rt_std = np.std(results_rt)
        
    return (dataset, sk_mean, rt_mean, sk_std, rt_std, sk_time, rt_time, metric)

In [16]:
results_reg = [evaluate_dataset(d, "reg") for d in datasets["reg"]]
results_clf = [evaluate_dataset(d, "clf") for d in datasets["clf"]]
results = results_reg + results_clf

cols = "dataset sk_mean rt_mean sk_std rt_std sk_time(s) rt_time(s) metric".split()

pd.DataFrame(results, columns=cols)


dataset   sk_mean   rt_mean        sk_std        rt_std  sk_time(s)  \
0       diabetes  0.312214  0.302783  3.762691e-02  3.623210e-02    0.002715   
1        housing  0.599732  0.599732  1.126753e-16  1.110223e-16    0.043528   
2  breast_cancer  0.929053  0.928561  6.796157e-03  6.026922e-03    0.004432   
3        titanic  0.786441  0.779661  1.110223e-16  3.330669e-16    0.002504   

   rt_time(s) metric  
0    0.020583     r2  
1    0.826759     r2  
2    0.053104    acc  
3    0.019823    acc

In [17]:
results_reg = [evaluate_dataset(d, "reg", model="rf", n_repeats=10) for d in datasets["reg"]]
results_clf = [evaluate_dataset(d, "clf", model="rf", n_repeats=10) for d in datasets["clf"]]
results = results_reg + results_clf

cols = "dataset sk_mean rt_mean sk_std rt_std sk_time(s) rt_time(s) metric".split()

pd.DataFrame(results, columns=cols)


dataset   sk_mean   rt_mean    sk_std    rt_std  sk_time(s)  \
0       diabetes  0.431703  0.447304  0.027677  0.028459    0.034850   
1        housing  0.643526  0.642669  0.003052  0.002625    0.091276   
2  breast_cancer  0.948772  0.936842  0.012475  0.005436    0.032648   
3        titanic  0.801356  0.789492  0.008331  0.010979    0.035068   

   rt_time(s) metric  
0    0.050684     r2  
1    2.235815     r2  
2    0.199760    acc  
3    0.067969    acc